# Zustandsreglerentwurf

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import sympy as sp
from IPython.display import display, Image, Latex, Markdown
from scipy.integrate import solve_ivp

In [ ]:
imag = Image("../images/zustandsregler_beobachter_vorsteuerung.png", width=600)
display(imag)